In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool
from azure.identity import DefaultAzureCredential


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [2]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [3]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [4]:
from azure.ai.projects.models import (
    AsyncToolSet,
    BingGroundingTool,
    CodeInterpreterTool
)

toolset = AsyncToolSet()

# Add the code interpreter tool
code_interpreter = CodeInterpreterTool()
toolset.add(code_interpreter)

API_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT")

In [5]:
async def data_science_ai_agent(query: str) -> str:
    print("This is caculator for Azure AI Agent Service .......")
    
    with project_client:
        agent = project_client.agents.create_agent(
            model=API_DEPLOYMENT_NAME,
            name="Data Science AI Agent",
            instructions="Your role is to assist users with  data compare. using the code interpreter the process data when there are some numbers, for compare two number, you can use python to compare which one is bigger. and to retry if errors occur.",
            toolset=toolset,
            temperature=0.2,
            headers={"x-ms-enable-preview": "true"},
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [6]:
calculator_agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[data_science_ai_agent],
    system_message="Use tools to solve tasks.",
)

In [7]:
async def assistant_run() -> None:
    response = await calculator_agent.on_messages(
            [TextMessage(content="which one is bigger? 9.9 or 9.11 ", source="user")],
            cancellation_token=CancellationToken(),
    )
#     print(response.inner_messages)
    print(response.chat_message)

In [8]:
await assistant_run()

source='assistant' models_usage=RequestUsage(prompt_tokens=68, completion_tokens=12) content='9.11 is bigger than 9.9.' type='TextMessage'
